In [1]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion


In [161]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [20]:
import pandas as pd
from glob import glob

files = glob("../data/test*pred.csv")

dfs = [pd.read_csv(f) for f in files]


df_test = pd.concat(dfs)


## Error analysis

In [23]:
from rioplatense_hs.preprocessing import text_to_label, labels
from sklearn.metrics import classification_report

pred_labels = [f"PRED_{label}" for label in labels]
gold_labels = df_test[labels]
preds = df_test[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.16      0.29      0.21        42
       LGBTI       0.48      0.67      0.56        33
      RACISM       0.57      0.72      0.64       101
       CLASS       0.27      0.38      0.32        32
    POLITICS       0.21      0.53      0.30        53
    DISABLED       0.38      0.31      0.34        29
  APPEARANCE       0.44      0.55      0.49        75
    CRIMINAL       0.31      0.44      0.36        64

   micro avg       0.36      0.52      0.42       429
   macro avg       0.35      0.48      0.40       429
weighted avg       0.38      0.52      0.44       429
 samples avg       0.08      0.08      0.08       429



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
df_test["num_hatred"] = df_test[labels].sum(axis=1)

In [25]:
pred_labels = [l for l in df_test.columns if l.startswith("PRED")]

label_hate = df_test[labels].sum(axis=1) > 0
pred_hate = df_test[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.93      0.85      0.89      1932
        True       0.42      0.63      0.50       336

    accuracy                           0.81      2268
   macro avg       0.67      0.74      0.69      2268
weighted avg       0.85      0.81      0.83      2268



In [27]:
# Just keep the most important characteristics
core_labels = ["WOMEN", "LGBTI", "RACISM", "CLASS"]
pred_core_labels = [f"PRED_{label}" for label in core_labels]

label_core_hate = df_test[core_labels].sum(axis=1) > 0
pred_core_hate = df_test[pred_core_labels].sum(axis=1) > 0

print(classification_report(label_core_hate, pred_core_hate))


              precision    recall  f1-score   support

       False       0.97      0.94      0.95      2097
        True       0.45      0.64      0.53       171

    accuracy                           0.91      2268
   macro avg       0.71      0.79      0.74      2268
weighted avg       0.93      0.91      0.92      2268



In [17]:
pd.options.display.max_colwidth = 1000

df_test[pred_hate & ~label_hate][["context_tweet", "text", "pred_cot", ]].sample(10)

,context_tweet,text,pred_cot
141,La sorprendente jura de una egresada universitaria: tachó Patria y puso Matria https://t.co/NjgZVVQJ2y https://t.co/txZNslH8N1,"@usuario Yo le quitaría el título por el simple hecho de no entender que La Patria es ""femenino""\n\nPor otra parte, en mi empresa ni en pedo estamos contratando a este tipo de gente... ¡Es quilombo labural seguro!","El texto critica a la egresada universitaria por cambiar el término ""Patria"" por ""Matria"", y además menciona que en su empresa no contratarían a este tipo de gente, lo cual alude a su ideología política y genera rechazo hacia el grupo al que pertenece. La respuesta final es ""política""."
714,Evo Morales aseguró desde Buenos Aires que Luis Arce ganó las elecciones en Bolivia https://t.co/puEDB1K5IQ,@usuario Que asco este pedófilo es una cloaca este país,"El texto alude a una acusación grave y sin fundamentos de pedofilia hacia Evo Morales. La respuesta final es ""política""."
786,"Descontrol en Año Nuevo: sin tapabocas ni distancia social, miles de jóvenes festejaron en parques y plazas de CABA https://t.co/t8gbsOVBbk",@usuario Dejen vivir! Cómplices de la diktadura sanitaria!,"El comentario critica las medidas sanitarias y llama ""cómplices de la dictadura sanitaria"" a quienes respetan esas medidas. La respuesta final es ""política""."
828,El PJ elogió la actitud de China frente a la pandemia del coronavirus https://t.co/IYGVlArQuJ,@usuario Que lindo yo pedí chop suey! Miro Bruce Lee y espero las ofertas del súper chino de la esquina!🈯🎎🎴 #bastadesopademurcielago,"El texto hace alusiones estereotipadas hacia los chinos y su cultura. La respuesta final es ""racismo"""
1005,Joe Biden presentó su nuevo equipo de comunicación compuesto enteramente por mujeres https://t.co/8Pla4Vxxkw,"@usuario Wow, tan feminista que ya es machista","El texto critica a Biden por su elección de un equipo de comunicación compuesto enteramente por mujeres, acusándolo de ser ""machista"". La respuesta final es ""mujer, política""."
936,Pablo Moyano amenazó con un paro en la Ciudad de Buenos Aires si hay recortes en la recolección de basura https://t.co/GnCtWaTCVn,@usuario Este 💩 q no se meta con la ciudad. Q se jodan por apoyar la reduccion de fondos esto lo hace este 👨🏿 para desgastar a Larreta. Que no sueñen con la Capital jamás será peronista NO QUEREMOS CEREBROS PLANOS EN LA CIUDAD AUTÓNOMA DE BUENOS AIRES. para eso esta la pba,"El texto critica a Pablo Moyano y utiliza un emoji de color de piel oscuro (""👨🏿"") como una forma de despreciarlo. Además, menciona que la Capital nunca será peronista, lo cual alude a una diferencia política. La respuesta final es ""política, aspecto""."
588,Su novia es mexicana y Migraciones le exige casi 50 mil pesos para dejarla ingresar a la Argentina | Por Cinthia Ruth https://t.co/dnNHPbdo7m,"@usuario Flaco, buscate alguna parecida de por acá y dejate de joder. Debe haber 10.000 iguales","El texto alude a que el hombre debería buscar una pareja similar en Argentina en lugar de traer a su novia mexicana. La respuesta final es ""racismo""."
513,"Juana Viale: ""¿Termina este mandato el Gobierno?"" https://t.co/hGmInRVN8j",@usuario Juanita Viale. Espécimen que existe gracias a su abuela que está hace 104 años y que de paso se daba con Lousteau estando embarazada de su marido. Impresentable. Le damos entidad a esta soreta. Así estamos.,"El texto alude al aspecto de Juana Viale y critica su existencia relacionándola con su abuela y su vida personal. La respuesta final es ""aspecto"""
1115,"Perfil: quién es Brenda Madrid, la jueza suplente que sobreseyó a Pablo Moyano https://t.co/npQBNktuix https://t.co/BLaMzSO17U","@usuario Eso es jueza ?. La facha la vende , no se puede esperar otra cosa !","El comentario se refiere a la apariencia física de la jueza, aludiendo a su ""facha"". La respuesta final es ""aspecto""."
225,El Frente de Izquierda propone nuevos impuestos sobre patrimonios y sobre la propiedad de la tierra https://t.co/i9E7Vc9JF5 https:

In [78]:
df_train[df_train["num_hatred"] == 0][["context_tweet", "text", "text_label"]].sample(10)

,context_tweet,text,text_label
15354,Loly Antoniale mostró su impresionante casa en Miami: “Soy la reina de mi castillo” https://t.co/GuFd1ythLA,@usuario Mujeres hablando de mujeres 💜,nada
16828,La nueva vida en la cárcel de los rugbiers acusados de matar a Fernando Báez Sosa: comparten todos el mismo celular y leen novelas de fantasía https://t.co/VHfUewstWx,@usuario Uno piensa que solo en Colombia la justicia es ciega \n\nPero con este caso me doy cuenta que es en todo el mundo,nada
27859,"Violó a su nieta, recibió la prisión domiciliaria con la excusa del coronavirus y volvió a contactar a la víctima por teléfono https://t.co/Fgc5jKto1U","@usuario La quería poner el hombre, se solidarizó el juez.",nada
34678,"El equipo médico que venció al coronavirus en China recomendó endurecer la cuarentena en Italia: “Hay demasiada gente en la calle, no sé qué están pensando” https://t.co/CKfSJEuwIL",@usuario A mi me gustaría saber que mierda estaban pensando ellos cuando comían en esos mercados con todos los animales hacinados!,nada
1925,Agrupaciones feministas y la ministra de las Mujeres y Géneros se expresaron contra la liberación de violadores y femicidas https://t.co/ccZOTF32Qp,"@usuario Tu gobierno, querida.",nada
35235,"La asesina serial que mataba hombres para divertirse: sangre, sexo sadomasoquista y excitación en cada puñalada | Por Carolina Balbiani https://t.co/T3EdKxXmkf",@usuario Que rica,nada
25050,Organizaciones sociales cortan la avenida 9 de Julio: reclaman un salario mínimo de $ 45.000 https://t.co/DvQoh4SPaF,@usuario pasaje de ida a Haití y en 10 años somos potencia,nada
26016,"Cuarentena. Carrió: ""Si allanan a Macri, tienen que detener a Alberto Fernández "" https://t.co/GBNki8ew1D https://t.co/tMZwEurVa0",@usuario Bamos LILITA ES ASI,nada
589,Trump volvió a culpar a China por la pandemia y aseguró que “van a pagar un alto precio por lo que le hicieron a EEUU y al mundo” https://t.co/ZvfPtpZNPe,@usuario Te amo,nada
12317,"Juana Viale: ""Me afectó muchísimo que me dijeran golpista "" https://t.co/xNUKlEj9B9 https://t.co/GnejLiVYUB",@usuario Golpista y más aún. Viene de familia (de garcas),nada


In [104]:
# LGBTI errors

label = "CLASS"
sample_df.loc[sample_df[label] != sample_df[f"PRED_{label}"]][["context_tweet", "text", "text_label", "pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
64,🔴 AHORA | Alberto Fernández criticó a Susana Giménez y defendió a Marcelo Tinelli https://t.co/vMUxU3bdKB,@usuario Pregunta... los de la Villa Azul y sus alrededores no son Noticias? No está Berni ahí?,nada,"El texto menciona a la ""Villa Azul"", que es una referencia a un barrio de bajos recursos, y hace alusión a la presencia del Ministro de Seguridad de la provincia de Buenos Aires, Sergio Berni. Por lo tanto, se está haciendo una asociación entre la villa y delincuencia. La respuesta final es ""clase, criminal""."
522,"El drama de una argentina en Bolivia: está embarazada de seis meses, duerme en la calle con su hijo de 4 años y no puede regresar al país | Por Florencia Illbele https://t.co/emWE3AxUqe",@usuario Que feo pensar que acá los bolivianos hasta planes sociales tiene y disfrutan de nuestro sistema de salud y educativo cada vez me convenzo más que los arg somos demasiados generosos,racismo,"El texto alude a los bolivianos y los critica por recibir planes sociales y disfrutar del sistema de salud y educación argentino. La respuesta final es ""racismo, clase""."
382,Coronavirus en Argentina: detienen a dos funcionarios de la Aduana que llevaban dinero de planes sociales argentinos a Paraguay https://t.co/HtxVVz1oZ2,"@usuario AHHHH !!! QUE SON KK ,LOS MUCHACHOS ,,JEJEEJ, ARGENTINA PAIS DE PELOTUDOS /AS..",nada,"El texto utiliza expresiones despectivas y palabras ofensivas para referirse a los funcionarios y a los argentinos en general. La respuesta final es ""clase"""
427,"Relato de un aborto en medio de la desesperación: “Me jugué la vida, cuando no tenés plata te la jugás” | Por Gisele Sousa Dias https://t.co/kFFKKuARi8",@usuario Todo por no ponerse 1 forro! Tomatelas PAYASA,nada,"El texto critica a la persona por no haber utilizado un condón y la insulta llamándola ""payasa"". La respuesta final es ""clase""."
66,"Coronavirus en Argentina: 341 presos bonaerenses inician huelga de hambre y piden ""irse a sus casas"" https://t.co/GXtHXFL6te","@usuario Un gasto menos, dinero q puede ser invertido en salud. Y Si llegan a quemar colchones en señal d protesta como es su costumbre, q duerman en el piso!",nada,"El texto se refiere a los presos como un gasto innecesario y sugiere que si llegan a protestar quemando colchones, deberían dormir en el piso. La respuesta final es ""criminal, clase""."


In [80]:
df_train[(df_train["POLITICS"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
31940,"Grabois defendió la toma de tierras: ""El que dice que ocupar es delito, miente "" https://t.co/qXZsHYnmZ3 https://t.co/R2BLqqV23a",@usuario Yo le pongo una dinamita a estos vagos,"clase, política"
16559,"Le sacó el arma al ladrón que lo asaltó, lo corrió y lo mató de un disparo: quedó detenido https://t.co/fU6UWLDqQe","@usuario Cagones hoy marchen para que suelten a este hombre, no por el hijo de puta de Macri que nos fundió y ayudó a que vuelvan estos negros hijos de puta kirchnerista","racismo, política, criminal"
7604,Santa Fe: rompen más silobolsas y crece la preocupación en el campo https://t.co/Pb1gaBqyXU https://t.co/cPlGjy3V6T,@usuario Donde quedaron los escopetazos Argentina mia?...,"política, criminal"
34614,"Juan Grabois: ""Hay una oleada de toma de tierras y se va a profundizar "" https://t.co/2B1JL2BkHh https://t.co/jJXbT0X564","@usuario Si es una amenaza!!!!??!!...a Vos hdrmPtas, seras otro de los tantos que vamos a LINCHAR en la Plaza de Mayo!!!!","clase, política"
21056,"“Tenemos explotados los teléfonos con denuncias por gente que no cumple la cuarentena”, aseguró la titular de Migraciones | Por Hugo Martin https://t.co/IJgFwW4Gbk","@usuario Se supone q es gente escolarizada, con educación, después dicen peronchos! Hdp","racismo, política"


In [91]:
sample_df["PRED_HATEFUL"] = pred_hate.astype(int)

In [92]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)